In [1]:
import sys
sys.path.append("..") # Ensure the parent directory is in the path

import core.AssignmentTransformer as TransformerAnalysis
from core.DataLoader import DataPreprocessor
import core.AssignmentKFold as KFold
import numpy as np
from importlib import reload
import keras
import tensorflow as tf
import matplotlib.pyplot as plt
MAX_JETS = 4
PLOTS_DIR = f"plots/"
import os
if not os.path.exists(PLOTS_DIR):
    os.makedirs(PLOTS_DIR)

DataProcessor = DataPreprocessor(jet_features=["ordered_jet_pt", "ordered_jet_e", "ordered_jet_phi", "ordered_jet_eta", "ordered_jet_b_tag","m_l1j", "m_l2j", "dR_l1j", "dR_l2j"], 
                                lepton_features=["lep_pt","lep_e", "lep_eta", "lep_phi"],
                                jet_truth_label="ordered_event_jet_truth_idx", 
                                lepton_truth_label="event_lepton_truth_idx", 
                                global_features = ["met_met_NOSYS","met_phi_NOSYS"], 
                                max_leptons=2, 
                                max_jets = MAX_JETS, 
                                non_training_features= ["truth_ttbar_mass", "truth_ttbar_pt", "N_jets"], 
                                event_weight="weight_mc_NOSYS")
DataProcessor.load_data("/data/dust/group/atlas/ttreco/full_training.root", "reco", max_events=100000)
DataProcessor.prepare_data()
DataProcessor.normalise_data()
DataProcessor.split_data(0.2)

2025-10-05 14:01:09.614673: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-10-05 14:01:09.619773: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-10-05 14:01:09.633689: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1759665669.654522 3427471 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1759665669.660653 3427471 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1759665669.676997 3427471 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

In [2]:
reload(TransformerAnalysis)
reload(KFold)

TransformerMatcher = TransformerAnalysis.TransformerJetMatcher(DataProcessor); TransformerMatcher.load_data(*DataProcessor.get_data())
TransformerMatcher.build_model(
    hidden_size=16,
    lep_dim=4,
    global_dim=2,
    jet_embedding_layers=2,
    lepton_embedding_layers=2,
    global_embedding_layers=2,
    attention_blocks=2,
    ff_layers=3,
    n_heads=6,
    dropout_rate=0.1,
)
TransformerMatcher.compile_model(
    lambda_excl=0, optimizer=keras.optimizers.Adam(learning_rate=1e-4)
)
TransformerMatcher.model.summary()
TransformerMatcher.load_data(*DataProcessor.get_data())

New TransformerJetMatcher model initialized.


2025-10-05 14:01:27.760346: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Model: "TransformerJetMatcher"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ lepton (InputLayer) │ (None, 2, 4)      │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global (InputLayer) │ (None, 1, 2)      │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lepton_embedding_1  │ (None, 2, 4)      │         20 │ lepton[0][0]      │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_embedding_1  │ (None, 1, 2)      │          6 │ global[0][0]      │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ jet (InputLayer)    │ (None, 4, 9)      │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lepton_embedding_d… │ (None, 2, 4)      │          0 │ lepton_embedding… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_embedding_d… │ (None, 1, 2)      │          0 │ global_embedding… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ jet_embedding_1     │ (None, 4, 16)     │        160 │ jet[0][0]         │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lepton_embedding_2  │ (None, 2, 4)      │         20 │ lepton_embedding… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_embedding_2  │ (None, 1, 2)      │          6 │ global_embedding… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ jet_embedding_drop… │ (None, 4, 16)     │          0 │ jet_embedding_1[… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lepton_embedding_d… │ (None, 2, 4)      │          0 │ lepton_embedding… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_embedding_d… │ (None, 1, 2)      │          0 │ global_embedding… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ jet_embedding_2     │ (None, 4, 16)     │        272 │ jet_embedding_dr… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape_1 (Reshape) │ (None, 8)         │          0 │ lepton_embedding… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape (Reshape)   │ (None, 2)         │          0 │ global_embedding… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ jet_embedding_drop… │ (None, 4, 16)     │          0 │ jet_embedding_2[… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ repeat_vector_1     │ (None, 4, 8)      │          0 │ reshape_1[0][0]   │
│ (RepeatVector)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 16,966 (66.27 KB)

 Trainable params: 16,966 (66.27 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
TransformerMatcher.train_model(epochs=100,
                                batch_size=128,
                                verbose=1,
                                #weight = "sample",
                                callbacks = keras.callbacks.EarlyStopping(monitor="val_accuracy", patience=50, restore_best_weights=True, mode ="max"))


Starting training...
Epoch 1/100
622/622 ━━━━━━━━━━━━━━━━━━━━ 23s 22ms/step - accuracy: 0.6812 - loss: 2.1667 - val_accuracy: 0.7715 - val_loss: 1.7456
Epoch 2/100
622/622 ━━━━━━━━━━━━━━━━━━━━ 12s 19ms/step - accuracy: 0.7375 - loss: 1.8919 - val_accuracy: 0.7676 - val_loss: 1.5057
Epoch 3/100
622/622 ━━━━━━━━━━━━━━━━━━━━ 12s 19ms/step - accuracy: 0.7468 - loss: 1.6997 - val_accuracy: 0.7661 - val_loss: 1.3887
Epoch 4/100
622/622 ━━━━━━━━━━━━━━━━━━━━ 12s 19ms/step - accuracy: 0.7521 - loss: 1.5892 - val_accuracy: 0.7861 - val_loss: 1.2576
Epoch 5/100
622/622 ━━━━━━━━━━━━━━━━━━━━ 12s 20ms/step - accuracy: 0.7581 - loss: 1.4983 - val_accuracy: 0.7873 - val_loss: 1.1563
Epoch 6/100
622/622 ━━━━━━━━━━━━━━━━━━━━ 12s 20ms/step - accuracy: 0.7689 - loss: 1.3998 - val_accuracy: 0.8015 - val_loss: 1.0956
Epoch 7/100
622/622 ━━━━━━━━━━━━━━━━━━━━ 12s 19ms/step - accuracy: 0.7706 - loss: 1.3442 - val_accuracy: 0.8017 - val_loss: 1.0702
Epoch 8/100
622/622 ━━━━━━━━━━━━━━━━━━━━ 12s 19ms/step - accur

In [ ]:
TransformerMatcher.plot_history()

In [ ]:
TransformerMatcher.plot_confusion_matrix()

In [ ]:
TransformerMatcher.plot_permutation_importance(shuffle_number=5)